In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from sklearn.linear_model import LinearRegression
import pickle
# from Model_Build.Functions import *

In [36]:
rating_tables = pd.read_html('https://www.basketball-reference.com/leagues/NBA_2022_ratings.html')
rating_table = rating_tables[0]
rating_table.columns = rating_table.columns.droplevel(0)
rating_table['Games'] = rating_table.W + rating_table.L

In [37]:
team_map = {
    'LA Lakers' : 'Los Angeles Lakers',
    'Charlotte' : 'Charlotte Hornets',
    'Sacramento' : 'Sacramento Kings',
    'San Antonio' : 'San Antonio Spurs',
    'Houston' : 'Houston Rockets',
    'Minnesota' : 'Minnesota Timberwolves',
    'Phoenix' : 'Phoenix Suns',
    'Brooklyn' : 'Brooklyn Nets',
    'Detroit' : 'Detroit Pistons',
    'Memphis' : 'Memphis Grizzlies',
    'Milwaukee' : 'Milwaukee Bucks',
    'Boston' : 'Boston Celtics',
    'Utah' : 'Utah Jazz',
    'Golden State' : 'Golden State Warriors',
    'LA Clippers' : 'Los Angeles Clippers',
    'Orlando' : 'Orlando Magic',
    'Chicago' : 'Chicago Bulls',
    'Portland' : 'Portland Trail Blazers',
    'Okla City': 'Oklahoma City Thunder',
    'New Orleans' : 'New Orleans Pelicans',
    'Indiana' : 'Indiana Pacers',
    'Washington' : 'Washington Wizards',
    'Cleveland' : 'Cleveland Cavaliers',
    'Atlanta' : 'Atlanta Hawks',
    'Toronto' : 'Toronto Raptors',
    'Denver' : 'Denver Nuggets',
    'New York' : 'New York Knicks',
    'Philadelphia' : 'Philadelphia 76ers',
    'Miami' : 'Miami Heat',
    'Dallas' : 'Dallas Mavericks'
}
possesions_table = pd.read_html('https://www.teamrankings.com/nba/stat/possessions-per-game')
possesions_table = possesions_table[0]
possesions_table['Team'] = possesions_table.Team.apply(lambda x: team_map[x])
possesions_table = possesions_table[['Team', '2021']]
possesions_table.columns = ['Team', 'Possesions/Game']

In [58]:
merged = pd.merge(rating_table, possesions_table, on = 'Team', how = 'inner')
merged['Possesions'] = merged.Games*merged['Possesions/Game']
merged['Adj_Point_Differential'] = merged.Possesions/100 * merged.NRtg
merged['Adj_Point_Differential_82'] = merged.Adj_Point_Differential*(82/merged.Games)
merged['Adj_Point_Differential_82'] = merged.Adj_Point_Differential_82.apply(round, 0)
merged['Adj_Point_Differential_82'] = merged.Adj_Point_Differential_82.apply(int)
merged

,Rk,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,Games,Possesions/Game,Possesions,Adj_Point_Differential,Adj_Point_Differential_82
0,1,Golden State Warriors,W,P,29,7,0.806,9.83,113.30,103.41,9.90,9.62,113.12,103.46,9.66,36,102.0,3672.0,363.52800,828
1,2,Utah Jazz,W,NW,27,10,0.730,9.89,118.48,108.35,10.13,9.13,117.74,108.39,9.35,37,102.0,3774.0,382.30620,847
2,3,Phoenix Suns,W,P,28,8,0.778,7.50,112.67,105.30,7.37,7.12,112.55,105.57,6.98,36,103.3,3718.8,274.07556,624
3,4,Cleveland Cavaliers,E,C,21,16,0.568,5.32,111.77,106.35,5.42,5.73,111.59,105.75,5.84,37,100.3,3711.1,201.14162,446
4,5,Miami Heat,E,SE,23,15,0.605,3.71,112.95,109.13,3.82,3.24,113.20,109.84,3.36,38,99.4,3777.2,144.28904,311
5,6,Memphis Grizzlies,W,SW,24,14,0.632,3.16,112.96,109.81,3.14,3.15,112.86,109.73,3.13,38,102.7,3902.6,122.54164,264
6,7,Chicago Bulls,E,C,25,10,0.714,3.71,113.19,109.54,3.65,2.85,112.75,109.94,2.81,35,101.5,3552.5,129.66625,304
7,8,Milwaukee Bucks,E,C,25,14,0.641,4.31,113.39,109.06,4.33,2.71,112.98,110.24,2.74,39,102.7,4005.3,173.42949,365
8,9,Brooklyn Nets,E,A,23,12,0.657,2.83,111.54,108.64,2.90,1.96,111.60,109.57,2.03,35,103.1,3608.5,104.64650,245
9,10,Boston Celtics,E,A,18,19,0.486,1.14,109.97,108.82,1.15,1.57,110.25,108.66,1.58,37,102.3,3785.1,43.52865,96


In [ ]:
file_name = 'Model_Build/Data/win_pct_regression.pickle'
with open(file_name, 'rb') as f:
    model = pickle.load(f)